In [27]:
import pandas as pd
import preprocessor as p
import re
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tag.perceptron import PerceptronTagger

In [2]:
training_text = pd.read_csv('/Users/matthewcassi/Documents/nhl_sentiment_working/sentiment_data/training_sentiment.csv', encoding='ISO-8859-1', header=None)
testing_text = pd.read_csv('/Users/matthewcassi/Documents/nhl_sentiment_working/sentiment_data/testing_sentiment.csv', encoding='ISO-8859-1', header=None)

In [3]:
training_text.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
training_text.columns = ['sentiment','id','date','query','user','text']
testing_text.columns = ['sentiment','id','date','query','user','text']

In [5]:
training_text.head()

,sentiment,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
training_text = training_text[['sentiment','text']]
testing_text = testing_text[['sentiment','text']]

In [7]:
testing_text = testing_text[testing_text['sentiment'] != 2]

In [8]:
testing_text['sentiment']

0      4
1      4
2      4
3      4
4      4
5      4
6      0
7      4
8      4
9      4
11     0
12     4
13     4
14     0
15     4
16     0
17     4
18     0
19     4
20     4
21     4
22     4
23     4
24     4
25     4
26     4
27     4
28     4
29     4
32     4
      ..
457    0
458    0
461    0
462    0
463    4
464    0
467    4
468    0
469    4
470    4
471    4
474    0
476    4
477    4
478    4
479    4
480    0
481    0
482    0
484    0
485    0
487    4
489    4
490    0
491    4
492    4
494    0
495    4
496    0
497    0
Name: sentiment, Length: 359, dtype: int64

In [9]:
map_dict = {0:0, 4:1}

training_text['sentiment'] = training_text['sentiment'].map(map_dict)
testing_text['sentiment'] = testing_text['sentiment'].map(map_dict)

In [10]:
def html_fix(string):
    string = BeautifulSoup(string, 'lxml').get_text()
    return string

In [11]:
training_text['text'] = training_text['text'].apply(html_fix)
testing_text['text'] = testing_text['text'].apply(html_fix)

In [12]:
# row wise function to remove URLs and emojis. takes a row and a column
def remove_urls(row, col):
    # set tweet preprocessing to remove emojis and URLs
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.MENTION, p.OPT.HASHTAG)
    
    # remove emojis and URLs from the column
    if type(row[col]) == str:
        return p.clean(row[col])

In [13]:
training_text['text'] = training_text.apply(remove_urls, axis=1, col='text')
testing_text['text'] = testing_text.apply(remove_urls, axis=1, col='text')

In [14]:
training_text['text'] = training_text['text'].str.lower()
testing_text['text'] = testing_text['text'].str.lower()

In [15]:
# replace the rt with nothing at the front of all tweet_1 columns
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])rt(?![a-z])', '', case=False)
# replace hashtag with nothing
training_text['text'] = training_text['text'].str.replace('#', '', case=False)
# replace u/U with you
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])u(?![a-z])', 'you', case=False) 
# replace r with are
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace r with are
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace r with are
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace thx with thanks
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])thx(?![a-z])', 'thanks', case=False)
# replace ur with your
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ur(?![a-z])', 'your', case=False)
# replace bc with because
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])bc(?![a-z])', 'because', case=False)
# replace abt with about
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])abt(?![a-z])', 'about', case=False)
# replace txt with tweet_1
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])txt(?![a-z])', 'tweet_1', case=False)
# replace fb with facebook
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fb(?![a-z])', 'facebook', case=False)
# replace w with with
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])w(?![a-z])', 'with', case=False)
# replace b4 with before
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])b4(?![a-z])', 'before', case=False)
# replace ast with assist
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ast(?![a-z])', 'assist', case=False)
# replace tgif with thank god its friday
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tgif(?![a-z])', 'thank god its friday', case=False)
# replace ily with i love you
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ily(?![a-z])', 'i love you', case=False)
# replace irl with in real life
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])irl(?![a-z])', 'in real life', case=False)
# replace idk with i dont know
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])idk(?![a-z])', 'i dont know', case=False)
# replace ftw with for the win
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ftw(?![a-z])', 'for the win', case=False)
# replace idc with i dont care
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])idc(?![a-z])', 'i dont care', case=False)
# replace tbt with throwback thursday
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tbt(?![a-z])', 'throwback thursday', case=False)
# replace tbh with to be honest
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tbh(?![a-z])', 'to be honest', case=False)
# replace tfw with that face when
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tfw(?![a-z])', 'that face when', case=False)
# replace qotd with quote of the day
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])qotd(?![a-z])', 'quote of the day', case=False)
# replace bff with best friends forever
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])bff(?![a-z])', 'best friends forever', case=False)
# replace jk with just kidding
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])jk(?![a-z])', 'just kidding', case=False)
# replace smh with shaking my head
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])smh(?![a-z])', 'shaking my head', case=False)
# replace btw with by the way
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])btw(?![a-z])', 'by the way', case=False)
# replace imo with in my opinion
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])imo(?![a-z])', 'in my opinion', case=False)
# replace imho with in my honest opinion
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])imho(?![a-z])', 'in my honest opinion', case=False)
# replace icymi with in case you missed it
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])icymi(?![a-z])', 'in case you missed it', case=False)
# replace lol with laugh out loud
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])lol(?![a-z])', 'laugh out loud', case=False)
# replace fwiw with for what its worth
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fwiw(?![a-z])', 'for what its worth', case=False)
# replace afaik with as far as i know
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])afaik(?![a-z])', 'as far as i know', case=False)
# replace gg with good game
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])gg(?![a-z])', 'good game', case=False)
# replace fyi with for your information
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fyi(?![a-z])', 'for your information', case=False)
# replace num with number
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])num(?![a-z])', 'number', case=False)
# replace omg with oh my god
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])omg(?![a-z])', 'oh my god', case=False)
# replace omfg with oh my fucking god
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])omfg(?![a-z])', 'oh my fucking god', case=False)
# replace asap with as soon as possible
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])asap(?![a-z])', 'as soon as possible', case=False)
# replace til with today i learned
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])til(?![a-z])', 'today i learned', case=False)
# replace lmao with laugh my ass off
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])lmao(?![a-z])', 'laugh my ass off', case=False)
# replace lmfao with laugh my fucking ass off
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])lmfao(?![a-z])', 'laugh my fucking ass off', case=False)
# replace dm with direct message
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])dm(?![a-z])', 'direct message', case=False)
# replace rofl with rolling on the floor laughing
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])rofl(?![a-z])', 'rolling on the floor laughing', case=False)
# replace fomo with fear of missing out
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fomo(?![a-z])', 'fear of missing out', case=False)
# replace orly with oh really
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])orly(?![a-z])', 'oh really', case=False)
# replace eli5 with explain like im five
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])eli5(?![a-z])', 'explain like im five', case=False)
# replace tmi with too much information
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tmi(?![a-z])', 'too much information', case=False)
# replace &amp; with and
training_text['text'] = training_text['text'].str.replace('&amp;', 'and', case=False)
# replace yrs with years
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yrs(?![a-z])', 'years', case=False)
# replace yr with year
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yr(?![a-z])', 'year', case=False)
# replace pt with point
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pt(?![a-z])', 'point', case=False)
# replace pts with points
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace - with ' '
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace % with percent
training_text['text'] = training_text['text'].str.replace('%', 'percent', case=False)
# replace fireav with fire av
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fireav(?![a-z])', 'fire av', case=False)
# replace vs with versus
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace ir with injured reserve
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ir(?![a-z])', 'injured reserve', case=False)
# replace gp with game played
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])gp(?![a-z])', 'game played', case=False)
# replace pts with points
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace vip with very important person
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])vip(?![a-z])', 'very important person', case=False)
# replace pei with prince edward island
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pei(?![a-z])', 'prince edward island', case=False)
# replace hof with hall of fame
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])hof(?![a-z])', 'hall of fame', case=False)
# replace msg with madison square garden
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])msg(?![a-z])', 'madison square garden', case=False)
# replace #msg with madison square garden
training_text['text'] = training_text['text'].str.replace('#msg', 'madison square garden', case=False)
# replace pls with please
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pls(?![a-z])', 'please', case=False)

#contractions
training_text['text'] = training_text['text'].str.replace(r"won't", "will not", case=False)
training_text['text'] = training_text['text'].str.replace(r"can\'t", "can not", case=False)

# general
training_text['text'] = training_text['text'].str.replace(r"n\'t", " not", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'re", " are", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'s", " is", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'d", " would", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'ll", " will", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'t", " not", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'ve", " have", case=False)
training_text['text'] = training_text['text'].str.replace(r"\'m", " am", case=False)

In [16]:
# replace the rt with nothing at the front of all tweet_1 columns
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])rt(?![a-z])', '', case=False)
# replace hashtag with nothing
testing_text['text'] = testing_text['text'].str.replace('#', '', case=False)
# replace u/U with you
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])u(?![a-z])', 'you', case=False) 
# replace r with are
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace r with are
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace r with are
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])r(?![a-z])', 'are', case=False)
# replace thx with thanks
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])thx(?![a-z])', 'thanks', case=False)
# replace ur with your
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ur(?![a-z])', 'your', case=False)
# replace bc with because
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])bc(?![a-z])', 'because', case=False)
# replace abt with about
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])abt(?![a-z])', 'about', case=False)
# replace txt with tweet_1
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])txt(?![a-z])', 'tweet_1', case=False)
# replace fb with facebook
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fb(?![a-z])', 'facebook', case=False)
# replace w with with
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])w(?![a-z])', 'with', case=False)
# replace b4 with before
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])b4(?![a-z])', 'before', case=False)
# replace ast with assist
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ast(?![a-z])', 'assist', case=False)
# replace tgif with thank god its friday
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tgif(?![a-z])', 'thank god its friday', case=False)
# replace ily with i love you
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ily(?![a-z])', 'i love you', case=False)
# replace irl with in real life
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])irl(?![a-z])', 'in real life', case=False)
# replace idk with i dont know
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])idk(?![a-z])', 'i dont know', case=False)
# replace ftw with for the win
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ftw(?![a-z])', 'for the win', case=False)
# replace idc with i dont care
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])idc(?![a-z])', 'i dont care', case=False)
# replace tbt with throwback thursday
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tbt(?![a-z])', 'throwback thursday', case=False)
# replace tbh with to be honest
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tbh(?![a-z])', 'to be honest', case=False)
# replace tfw with that face when
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tfw(?![a-z])', 'that face when', case=False)
# replace qotd with quote of the day
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])qotd(?![a-z])', 'quote of the day', case=False)
# replace bff with best friends forever
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])bff(?![a-z])', 'best friends forever', case=False)
# replace jk with just kidding
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])jk(?![a-z])', 'just kidding', case=False)
# replace smh with shaking my head
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])smh(?![a-z])', 'shaking my head', case=False)
# replace btw with by the way
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])btw(?![a-z])', 'by the way', case=False)
# replace imo with in my opinion
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])imo(?![a-z])', 'in my opinion', case=False)
# replace imho with in my honest opinion
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])imho(?![a-z])', 'in my honest opinion', case=False)
# replace icymi with in case you missed it
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])icymi(?![a-z])', 'in case you missed it', case=False)
# replace lol with laugh out loud
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])lol(?![a-z])', 'laugh out loud', case=False)
# replace fwiw with for what its worth
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fwiw(?![a-z])', 'for what its worth', case=False)
# replace afaik with as far as i know
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])afaik(?![a-z])', 'as far as i know', case=False)
# replace gg with good game
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])gg(?![a-z])', 'good game', case=False)
# replace fyi with for your information
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fyi(?![a-z])', 'for your information', case=False)
# replace num with number
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])num(?![a-z])', 'number', case=False)
# replace omg with oh my god
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])omg(?![a-z])', 'oh my god', case=False)
# replace omfg with oh my fucking god
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])omfg(?![a-z])', 'oh my fucking god', case=False)
# replace asap with as soon as possible
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])asap(?![a-z])', 'as soon as possible', case=False)
# replace til with today i learned
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])til(?![a-z])', 'today i learned', case=False)
# replace lmao with laugh my ass off
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])lmao(?![a-z])', 'laugh my ass off', case=False)
# replace lmfao with laugh my fucking ass off
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])lmfao(?![a-z])', 'laugh my fucking ass off', case=False)
# replace dm with direct message
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])dm(?![a-z])', 'direct message', case=False)
# replace rofl with rolling on the floor laughing
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])rofl(?![a-z])', 'rolling on the floor laughing', case=False)
# replace fomo with fear of missing out
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fomo(?![a-z])', 'fear of missing out', case=False)
# replace orly with oh really
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])orly(?![a-z])', 'oh really', case=False)
# replace eli5 with explain like im five
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])eli5(?![a-z])', 'explain like im five', case=False)
# replace tmi with too much information
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tmi(?![a-z])', 'too much information', case=False)
# replace &amp; with and
testing_text['text'] = testing_text['text'].str.replace('&amp;', 'and', case=False)
# replace yrs with years
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])yrs(?![a-z])', 'years', case=False)
# replace yr with year
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])yr(?![a-z])', 'year', case=False)
# replace pt with point
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pt(?![a-z])', 'point', case=False)
# replace pts with points
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace - with ' '
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace % with percent
testing_text['text'] = testing_text['text'].str.replace('%', 'percent', case=False)
# replace fireav with fire av
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fireav(?![a-z])', 'fire av', case=False)
# replace vs with versus
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace ir with injured reserve
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ir(?![a-z])', 'injured reserve', case=False)
# replace gp with game played
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])gp(?![a-z])', 'game played', case=False)
# replace pts with points
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pts(?![a-z])', 'points', case=False)
# replace vip with very important person
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])vip(?![a-z])', 'very important person', case=False)
# replace pei with prince edward island
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pei(?![a-z])', 'prince edward island', case=False)
# replace hof with hall of fame
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])hof(?![a-z])', 'hall of fame', case=False)
# replace msg with madison square garden
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])msg(?![a-z])', 'madison square garden', case=False)
# replace #msg with madison square garden
testing_text['text'] = testing_text['text'].str.replace('#msg', 'madison square garden', case=False)
# replace pls with please
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pls(?![a-z])', 'please', case=False)

#contractions
testing_text['text'] = testing_text['text'].str.replace(r"won't", "will not", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"can\'t", "can not", case=False)

# general
testing_text['text'] = testing_text['text'].str.replace(r"n\'t", " not", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'re", " are", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'s", " is", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'d", " would", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'ll", " will", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'t", " not", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'ve", " have", case=False)
testing_text['text'] = testing_text['text'].str.replace(r"\'m", " am", case=False)

In [17]:
def emojis(row, col):
    text = row[col]
    if type(row[col]) == str:
        text = re.sub(r'[^a-zA-Z ]+', '', row[col])
    return text

In [18]:
training_text['text'] = training_text.apply(emojis, axis=1, col='text')
testing_text['text'] = testing_text.apply(emojis, axis=1, col='text')

In [19]:
training_text.head()

,sentiment,text
0,0,awww that is a bummer you shoulda got david c...
1,0,is upset that he can not update his facebook b...
2,0,i dived many times for the ball managed to sav...
3,0,my whole body feels itchy and like its on fire
4,0,no it is not behaving at all i am mad why am i...


In [20]:
testing_text.head()

,sentiment,text
0,1,i loooooooovvvvvveee my kindle not that the dx...
1,1,reading my kindle love it lee childs is good read
2,1,ok first assesment of the it fucking rocks
3,1,you will love your kindle i have had mine for ...
4,1,fair enough but i have the kindle and i think ...


In [21]:
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])machinehead(?![a-z])', 'machine head', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])badgood(?![a-z])', 'bad good', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])nightce(?![a-z])', 'night', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])laaammmeeee(?![a-z])', 'lame', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])lifejesus(?![a-z])', 'life jesus', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])acquiescin(?![a-z])', 'acquiescing', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ammoxxx(?![a-z])', '', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])daddii(?![a-z])', 'daddy', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])fastits(?![a-z])', 'fast its', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])iamjonathancook(?![a-z])', 'i am jonathan cook', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])thtt(?![a-z])', 'that', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ihazfoodinmytummy(?![a-z])', 'i have food in my stomach', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])okreal(?![a-z])', 'ok real', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])withdrawnunity(?![a-z])', 'withdrawn unity', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])schbag(?![a-z])', 'bag', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])aaaaaaaaaahhhhhhhh(?![a-z])', 'ah', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])taboooo(?![a-z])', 'taboo', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])onechancenoend(?![a-z])', 'one chance no end', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])studyso(?![a-z])', 'study so', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])biitch(?![a-z])', 'bitch', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])quotooh(?![a-z])', 'quote', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yeahmonkey(?![a-z])', 'yeah monkey', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])nightandstill(?![a-z])', 'night and still', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])rightp(?![a-z])', 'right', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])lalajenny(?![a-z])', 'lala jenny', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])alrightt(?![a-z])', 'all right', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tradespacers(?![a-z])', 'trade spacers', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])hotdof(?![a-z])', 'hotdog', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])watchis(?![a-z])', 'watches', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])poppystephpage(?![a-z])', 'poppy steph page', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])reeaaaally(?![a-z])', 'really', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])kikkoanddaryl(?![a-z])', 'kikko and daryl', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])npatience(?![a-z])', 'patience', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])studyingmeh(?![a-z])', 'studying', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])chillinmymoodiamok(?![a-z])', 'chilling my mood i amok', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])mwahahahahohononono(?![a-z])', 'laughing no', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tiredsee(?![a-z])', 'tired see', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])suckin(?![a-z])', 'sucking', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])quottee(?![a-z])', 'quote', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])nowanother(?![a-z])', 'now another', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])redpath(?![a-z])', 'red path', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])straightiner(?![a-z])', 'straightener', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tennantcast(?![a-z])', 'tennant cast', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tremorsthis(?![a-z])', 'tremors this', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])jokerjax(?![a-z])', 'joker jax', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])twiiiiter(?![a-z])', 'twitter', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])isdidnt(?![a-z])', 'is didnt', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])badabing(?![a-z])', 'easy', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])andbeergnite(?![a-z])', 'and beer goodnight', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])davepts(?![a-z])', 'dave points', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])timbeeeeeeeeeeerinsert(?![a-z])', 'timber insert', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])strssd(?![a-z])', 'stressed', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tweeped(?![a-z])', '', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])windshieldroof(?![a-z])', 'windshield roof', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])codeshare(?![a-z])', 'code share', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])twittercut(?![a-z])', 'twitter cut', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])skincolor(?![a-z])', 'skin color', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])phoneeeeeeeeeeeeee(?![a-z])', 'phone', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])assnoles(?![a-z])', 'assholes', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pleasecloseyoureyes(?![a-z])', 'please close your eyes', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yooooooooooou(?![a-z])', 'you', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])blueskied(?![a-z])', 'blue skied', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])entrepenuercom(?![a-z])', '', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])freezinggg(?![a-z])', 'freezing', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])rainfuelled(?![a-z])', 'rain fuelled', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tomorrroow(?![a-z])', 'tomorrow', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])boredthe(?![a-z])', 'bored the', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])mcspaming(?![a-z])', 'mcspaming', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])toywell(?![a-z])', 'toy well', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])nvrmd(?![a-z])', 'nevermind', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])didwhats(?![a-z])', 'did whats', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])ohyes(?![a-z])', 'oh yes', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])toogala(?![a-z])', 'too gala', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])scarletbe(?![a-z])', 'scarlet be', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])listenning(?![a-z])', 'listening', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])thothat(?![a-z])', 'though that', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])sleepnightshift(?![a-z])', 'sleeping night shift', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])universitis(?![a-z])', 'universities', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])beein(?![a-z])', 'beeing', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])frushi(?![a-z])', '', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])forgotgod(?![a-z])', 'forgot god', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])goim(?![a-z])', 'going', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])bluelillyscouk(?![a-z])', 'blue lilly scouk', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])sighrest(?![a-z])', 'sigh rest', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])conflicing(?![a-z])', 'conflicting', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tredtwo(?![a-z])', 'tred two', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])withwho(?![a-z])', 'with who', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])somethingyoud(?![a-z])', 'something you would', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])niecejulianna(?![a-z])', 'niece julianna', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])mmmyeah(?![a-z])', 'yeah', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])eberybody(?![a-z])', 'everybody', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pealedthe(?![a-z])', 'pealed the', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])summerrrrrrrrrr(?![a-z])', 'summer', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])torontolong(?![a-z])', 'toronto long', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])iiiit(?![a-z])', 'it', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])scaregood(?![a-z])', 'scsre good', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])blooowwws(?![a-z])', 'blows', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])sittingin(?![a-z])', 'sitting', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])hopingforrain(?![a-z])', 'hoping for rain', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])tonightahhhhhhhhhh(?![a-z])', 'tonight', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])vanillasky(?![a-z])', 'vanilla sky', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])thoroughgood(?![a-z])', 'thorough good', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])hopr(?![a-z])', 'hope', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yupppppppp(?![a-z])', 'yup', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])inconsitent(?![a-z])', 'inconsistent', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])promfuck(?![a-z])', 'prom fuck', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])pigout(?![a-z])', 'pig out', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])mombrag(?![a-z])', 'mom brag', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])nevveerr(?![a-z])', 'never', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])wasfound(?![a-z])', 'was found', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])increasy(?![a-z])', 'increase', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])buzzzzzzz(?![a-z])', 'buzz', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])laptopiphone(?![a-z])', 'laptop mobile phone', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])luvfinally(?![a-z])', 'love finally', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])yeahhbaby(?![a-z])', 'yeah baby', case=False)

In [22]:
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])cuz(?![a-z])', 'because', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])sayin(?![a-z])', 'saying', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])nuggetssss(?![a-z])', 'nuggets', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])csfullerton(?![a-z])', 'california state fullerton', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])obamapelosi(?![a-z])', 'obama pelosi', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])obamabidenpelosi(?![a-z])', 'obama biden pelosi', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])vp(?![a-z])', 'vice president', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])atoday(?![a-z])', 'a today', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])irancraziness(?![a-z])', 'iran craziness', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fcking(?![a-z])', 'fucking', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])furkin(?![a-z])', 'fucking', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])contactsdoes(?![a-z])', 'contacts does', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])zomg(?![a-z])', 'oh my god', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])yearsgreat(?![a-z])', 'years great', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])funfun(?![a-z])', 'fun fun', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fuin(?![a-z])', 'fun', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])superslow(?![a-z])', 'super slow', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])newswow(?![a-z])', 'news wow', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])baseballamericacom(?![a-z])', 'baseball america', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])phoneughh(?![a-z])', 'phone ugh', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tonite(?![a-z])', 'tonight', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])scarey(?![a-z])', 'scary', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])propre(?![a-z])', 'proper', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])voiceover(?![a-z])', 'voice over', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])recentlyreleased(?![a-z])', 'recently released', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])supermarketing(?![a-z])', 'super marketing', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])byalex(?![a-z])', 'by alex', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tv(?![a-z])', 'television', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])atebits(?![a-z])', 'ate bits', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])soooooo(?![a-z])', 'so', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])timewarner(?![a-z])', 'time warner', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])espnjus(?![a-z])', 'espn just', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])daybankrupt(?![a-z])', 'day bankrupt', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])dood(?![a-z])', 'dude', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])webdesign(?![a-z])', 'web design', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])finewhat(?![a-z])', 'fine what', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])loooooooovvvvvveee(?![a-z])', 'love', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])yeahhhhhhhhh(?![a-z])', 'yeah', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tommorow(?![a-z])', 'tomorrow', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])gokey(?![a-z])', 'go key', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])hilariouslaugh(?![a-z])', 'hilarious laugh', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])hq(?![a-z])', 'headquarter', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])wtf(?![a-z])', 'what the fuck', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])hd(?![a-z])', 'high definition', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])youverse(?![a-z])', 'you verse', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])announcment(?![a-z])', 'announcement', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])newedge(?![a-z])', 'new edge', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fck(?![a-z])', 'fuck', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ajaxjquery(?![a-z])', 'ajax jquery', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])eectations(?![a-z])', 'expectations', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])slowlythey(?![a-z])', 'slowly they', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fricken(?![a-z])', 'fucking', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])chartvisualization(?![a-z])', 'chart visualization', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])lovin(?![a-z])', 'loving', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])wthi(?![a-z])', 'with', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])sittercity(?![a-z])', 'sitter city', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])universitys(?![a-z])', 'universities', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])viewunited(?![a-z])', 'view united', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])goddam(?![a-z])', 'god damn', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])watchin(?![a-z])', 'watching', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])oughta(?![a-z])', 'ought to', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])insightfilled(?![a-z])', 'insight filled', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])socialentrepreneurship(?![a-z])', 'social entrepreneurship', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])lawnmowing(?![a-z])', 'lawn mowing', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])assesment(?![a-z])', 'assessment', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ummm(?![a-z])', 'um', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ui(?![a-z])', 'user interface', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])nooooooo(?![a-z])', 'no', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])rantsandraves(?![a-z])', 'rants and raves', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])goooood(?![a-z])', 'good', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])fav(?![a-z])', 'favorite', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])wftb(?![a-z])', 'what the fuck', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])movieit(?![a-z])', 'movie it', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])disruptionfred(?![a-z])', 'disruption fred', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])sooo(?![a-z])', 'so', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])pretenious(?![a-z])', 'pretentious', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])flockofseagullsweregeopoliticallycorrect(?![a-z])', 'flock of seagulls were geopolitically correct', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])surgerywishing(?![a-z])', 'surgery wishing', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])goodby(?![a-z])', 'goodbye', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])mmmmmfamily(?![a-z])', 'family', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])loooong(?![a-z])', 'long', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])ahhgot(?![a-z])', 'ah got', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])enuf(?![a-z])', 'enough', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])bball(?![a-z])', 'basketball', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])audiobook(?![a-z])', 'audio book', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])hahaha(?![a-z])', 'laughing', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])discomort(?![a-z])', 'discomfort', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])soooo(?![a-z])', 'so', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])dentistwho(?![a-z])', 'dentist who', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])tattoooos(?![a-z])', 'tattoos', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])itwill(?![a-z])', 'it will', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])omgg(?![a-z])', 'oh my god', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])stormweight(?![a-z])', 'storm weight', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])bitin(?![a-z])', 'biting', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])monsta(?![a-z])', 'monster', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])studing(?![a-z])', 'studying', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])programme(?![a-z])', 'program', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])braindead(?![a-z])', 'brain dead', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])viralvideo(?![a-z])', 'viral video', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])resultsby(?![a-z])', 'results by', case=False)

In [23]:
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])shoulda(?![a-z])', 'should have', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])baack(?![a-z])', 'back', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])todaybut(?![a-z])', 'today but', case=False)
testing_text['text'] = testing_text['text'].str.replace('quot', '', case=False)
testing_text['text'] = testing_text['text'].str.replace(r'(?<![a-z])e(?![a-z])', 'quote', case=False)

training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])shoulda(?![a-z])', 'should have', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])baack(?![a-z])', 'back', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])todaybut(?![a-z])', 'today but', case=False)
training_text['text'] = training_text['text'].str.replace('quot', '', case=False)
training_text['text'] = training_text['text'].str.replace(r'(?<![a-z])e(?![a-z])', 'quote', case=False)

In [28]:
def remove_stops(data_str):
    # expects a string
    stops = set(stopwords.words("english"))
    list_pos = 0
    cleaned_str = ''
    text = data_str.split()
    for word in text:
        if word not in stops:
            # rebuild cleaned_str
            if list_pos == 0:
                cleaned_str = word
            else:
                cleaned_str = cleaned_str + ' ' + word
            list_pos += 1
    return cleaned_str

#def tag_and_remove(data_str):
#    cleaned_str = ' '
#    # noun tags
#    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
#    # adjectives
#    jj_tags = ['JJ', 'JJR', 'JJS']
#    # verbs
#    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
#    #adverbs
#    av_tags = ['RB','RBR','RBS']
#    nltk_tags = nn_tags + jj_tags + vb_tags + av_tags
#
#    # break string into 'words'
#    text = data_str.split()
#
#    # tag the text and keep only those with the right tags
#    #tagged_text = pos_tag(text)
#    tagged_text = tagger.tag(text)
#    for tagged_word in tagged_text:
#        if tagged_word[1] in nltk_tags:
#           cleaned_str += tagged_word[0] + ' '
#
#   return cleaned_str

def tag_and_remove(data_str):
    cleaned_str = ' '
    # noun tags
    nn_tags = ['NN', 'NNP', 'NNP', 'NNPS', 'NNS']
    # adjectives
    jj_tags = ['JJ', 'JJR', 'JJS']
    # verbs
    vb_tags = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
    #adverbs
    av_tags = ['RB','RBR','RBS']
    nltk_tags = nn_tags + jj_tags + vb_tags + av_tags

    # break string into 'words'
    text = data_str.split()

    # tag the text and keep only those with the right tags
    #tagged_text = pos_tag(text)
    tagged_text = tagger.tag(text)
    for tagged_word in tagged_text:
        if tagged_word[1] in nltk_tags:
            cleaned_str += tagged_word[0] + ' '

    return cleaned_str

def lemmatize(data_str):
    # expects a string
    list_pos = 0
    cleaned_str = ''
    lmtzr = WordNetLemmatizer()
    text = data_str.split()
    #tagged_words = pos_tag(text)
    tagged_words = tagger.tag(text)
    for word in tagged_words:
        if 'v' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='v')
        elif 'r' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='r')
        elif 'a' in word[1].lower():
            lemma = lmtzr.lemmatize(word[0], pos='a')
        else:
            lemma = lmtzr.lemmatize(word[0], pos='n')
        if list_pos == 0:
            cleaned_str = lemma
        else:
            cleaned_str = cleaned_str + ' ' + lemma
        list_pos += 1
    return cleaned_str


In [25]:
training_text['stop_text'] = training_text['text'].apply(remove_stops)
testing_text['stop_text'] = testing_text['text'].apply(remove_stops)

In [29]:
tagger = PerceptronTagger()
training_text['tag_text'] = training_text['stop_text'].apply(tag_and_remove)
testing_text['tag_text'] = testing_text['stop_text'].apply(tag_and_remove)

In [30]:
training_text['lemm_text'] = training_text['tag_text'].apply(lemmatize)
testing_text['lemm_text'] = testing_text['tag_text'].apply(lemmatize)

In [31]:
training_text['lemm_text']

0                       awww bummer get david carr third day
1          upset update facebook texting cry result schoo...
2          dive many time ball manage save percent rest g...
3                                 whole body feel itchy fire
4                                             behave mad see
5                                                 whole crew
6                                                   need hug
7          hey long time see yes rain bite bit laugh loud...
8                                                       nope
9                                                  que muera
10                           spring break plain city snowing
11                                             repierced ear
12                     bear watch think ua loss embarrassing
13                                count dont know never talk
14                   first gun really zac snyder doucheclown
15                              wish get watch miss premiere
16         hollis death 

In [32]:
training_text['stop_text']

0                       awww bummer got david carr third day
1          upset update facebook texting might cry result...
2          dived many times ball managed save percent res...
3                           whole body feels itchy like fire
4                                           behaving mad see
5                                                 whole crew
6                                                   need hug
7          hey long time see yes rains bit bit laugh loud...
8                                                       nope
9                                                  que muera
10                           spring break plain city snowing
11                                            repierced ears
12             could bear watch thought ua loss embarrassing
13                counts dont know either never talk anymore
14         would first gun really though zac snyder douch...
15                              wish got watch miss premiere
16         hollis death 

In [33]:
training_text[['sentiment','stop_text']].to_csv('stop_training.csv')
testing_text[['sentiment','stop_text']].to_csv('stop_testing.csv')

training_text[['sentiment','lemm_text']].to_csv('lemm_training.csv')
testing_text[['sentiment','lemm_text']].to_csv('lemm_testing.csv')

training_text[['sentiment','text']].to_csv('nothing_training.csv')
testing_text[['sentiment','text']].to_csv('nothing_testing.csv')

In [34]:
training_text.head()

,sentiment,text,stop_text,tag_text,lemm_text
0,0,awww that is a bummer you should have got dav...,awww bummer got david carr third day,awww bummer got david carr third day,awww bummer get david carr third day
1,0,is upset that he can not update his facebook b...,upset update facebook texting might cry result...,upset update facebook texting cry result scho...,upset update facebook texting cry result schoo...
2,0,i dived many times for the ball managed to sav...,dived many times ball managed save percent res...,dived many times ball managed save percent re...,dive many time ball manage save percent rest g...
3,0,my whole body feels itchy and like its on fire,whole body feels itchy like fire,whole body feels itchy fire,whole body feel itchy fire
4,0,no it is not behaving at all i am mad why am i...,behaving mad see,behaving mad see,behave mad see


In [35]:
testing_text.head(100)

,sentiment,text,stop_text,tag_text,lemm_text
0,1,i love my kindle not that the dx is cool but t...,love kindle dx cool fantastic right,love kindle dx cool fantastic right,love kindle dx cool fantastic right
1,1,reading my kindle love it lee childs is good read,reading kindle love lee childs good read,reading kindle love lee childs good read,read kindle love lee child good read
2,1,ok first assessment of the it fucking rocks,ok first assessment fucking rocks,ok first assessment fucking rocks,ok first assessment fucking rock
3,1,you will love your kindle i have had mine for ...,love kindle mine months never looked back new ...,love kindle mine months never looked new big ...,love kindle mine month never look new big huge...
4,1,fair enough but i have the kindle and i think ...,fair enough kindle think perfect,fair enough kindle think perfect,fair enough kindle think perfect
5,1,no it is too big i am quite happy with the kindle,big quite happy kindle,big quite happy kindle,big quite happy kindle
6,0,fuck this economy i hate aig and their non loa...,fuck economy hate aig non loan given asses,fuck economy hate aig non loan given asses,fuck economy hate aig non loan give ass
7,1,jquery is my new best friend,jquery new best friend,jquery new best friend,jquery new best friend
8,1,loves twitter,loves twitter,loves twitter,love twitter
9,1,how can you not love obama he makes jokes abou...,love obama makes jokes,love obama makes jokes,love obama make joke
